# SEMÁFOROS

In [2]:
import glob
import os
import sys
import carla
import argparse
import random
import time
import math
import numpy as np
import cv2

#### CREAR MUNDO

In [3]:
client = carla.Client('localhost', 2000)
client.set_timeout(2000)
#client.load_world("Town05")
world = client.get_world()
map = world.get_map()

#### CAMBIO DE VISTA

In [4]:
loc2=carla.Transform(carla.Location(x=64.557922, y=191.525208, z=1.138501), carla.Rotation(pitch=0.533207, yaw=-177.194565, roll=0.006472))
spectator = world.get_spectator()
spectator_pos = carla.Transform(loc2.location + carla.Location(x=10,y=0,z=4),carla.Rotation(yaw=loc2.rotation.yaw))
spectator.set_transform(spectator_pos)

#### CREACIÓN DEL VEHÍCULO

In [5]:
blueprint_library = world.get_blueprint_library()
bp = random.choice(blueprint_library.filter('vehicle.tesla.model3'))

if bp.has_attribute('color'):
    color = bp.get_attribute('color').recommended_values[2]
    bp.set_attribute('color', '255,0,255')

vehicle = world.spawn_actor(bp, loc2)

#### VELOCIDAD CONSTANTE

In [6]:
def maintain_speed(s,preferred_speed,speed_threshold):
    #el return es la aceleración
    if s>=preferred_speed:
        return 0
    elif s < preferred_speed - speed_threshold:
        return 0.8
    else:
        return 0.4

#### ANGULO

In [7]:
def angle_between(v1, v2):
    return math.degrees(np.arctan2(v1[1], v1[0]) - np.arctan2(v2[1], v2[0]))

def get_angle(car,wp):
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    
    car_vector = vehicle_pos.get_forward_vector()
    degrees = angle_between((x,y),(car_vector.x,car_vector.y))
    return degrees

In [8]:
#Calculo del giro
def steer(predicted_angle,MAX_STEER_DEGREES):
    if predicted_angle<-180:
        predicted_angle = predicted_angle+360
    elif predicted_angle > 180:
        predicted_angle = predicted_angle-360
    steer_input = predicted_angle
    if predicted_angle<-MAX_STEER_DEGREES:
        steer_input = -MAX_STEER_DEGREES
    elif predicted_angle>MAX_STEER_DEGREES:
        steer_input = MAX_STEER_DEGREES
    steer_input = steer_input/75
    return steer_input

#### SEMÁFORO

In [10]:
def is_traffic_light_red(vehicle, world):
    location = vehicle.get_location()
    actor_list = world.get_actors()

    for actor in actor_list:
        if actor.type_id.startswith("traffic.traffic_light"):
            traffic_light_location = actor.get_location()
            distance = location.distance(traffic_light_location)
            if distance < 37.0:
                traffic_light_state = actor.get_state()
                vehicle_waypoint = world.get_map().get_waypoint(location)
                traffic_light_waypoint = world.get_map().get_waypoint(traffic_light_location)
                if traffic_light_waypoint.lane_id == 1:
                    if traffic_light_state == carla.TrafficLightState.Red:
                        return True
                    elif traffic_light_state == carla.TrafficLightState.Green:
                        return False

#### FRENO

In [11]:
def freno(light,brake_intensity,vehicle,estimated_throttle,steer_input):
    if light:
        control = carla.VehicleControl(brake=brake_intensity)
        vehicle.apply_control(control)
    else:    
        vehicle.apply_control(carla.VehicleControl(throttle= estimated_throttle, steer=steer_input))

#### CONDUCCIÓN

In [13]:
MAX_STEER_DEGREES = 50
brake_intensity = 0.8 
preferred_speed = 30
speed_threshold = 2

while True:
    loc=vehicle.get_location()
    current_w = map.get_waypoint(loc)
    waypoint_separation = 5
    next_w0 = list(current_w.next(waypoint_separation))[0]
    next_w = map.get_waypoint(next_w0.transform.location,project_to_road=True, lane_type=(carla.LaneType.Driving))
    
    velocity = vehicle.get_velocity()
    speed = round(3.6 *math.sqrt(velocity.x**2+ velocity.y**2+velocity.z**2),0)
    estimated_throttle = maintain_speed(speed,preferred_speed,speed_threshold)
    predicted_angle = get_angle(vehicle,next_w)
    steer_input=steer(predicted_angle,MAX_STEER_DEGREES)
    
    light=is_traffic_light_red(vehicle, world)
    freno(light,brake_intensity,vehicle,estimated_throttle,steer_input)

KeyboardInterrupt: 

#### DESTRUIR

In [14]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()